In [73]:
import cv2
import os
import numpy as np
import pickle

In [74]:
# Image Enhancing

def preprocess_img(img):
    resized_img = cv2.resize(img, (256,256))

    gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)

    clahe = cv2.createCLAHE(3, (8,8))
    equalized_img = clahe.apply(gray)

    blurred_img = cv2.medianBlur(equalized_img, 5)

    # edges = cv2.Canny(blurred_img, 60, 90)
    # cv2.imshow('img', blurred_img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    hsv_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2HSV)
    hsv_img = cv2.medianBlur(hsv_img, 5)
    hist_h = cv2.calcHist([hsv_img], [0], None, [180], [0, 180]) 
    hist_s = cv2.calcHist([hsv_img], [1], None, [256], [0, 256])
    hist_v = cv2.calcHist([hsv_img], [2], None, [256], [0, 256])

    hist_h = hist_h / np.linalg.norm(hist_h)
    hist_s = hist_s / np.linalg.norm(hist_s)
    hist_v = hist_v / np.linalg.norm(hist_v)

    color_hist = np.concatenate([hist_h, hist_s, hist_v]).flatten()

    return blurred_img, color_hist

target_img = cv2.imread('./rotten-apple-13258722.webp')

final_target, target_hist = preprocess_img(target_img)

# cv2.imshow('final',final_target)
# cv2.waitKey(0)



In [75]:
# Feature Detection

sift = cv2.SIFT_create()

def extract_features(img):
    kp_sift, des_sift = sift.detectAndCompute(img, None)
    
    return kp_sift, des_sift

target_kp_sift, target_des_sift = extract_features(final_target)

target_desc = np.float32(target_des_sift)

# all_keypoints = kp_sift + kp_orb
    
# img_with_keypoints = cv2.drawKeypoints(target_img, target_kp_orb, None, color=(0,255,0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# cv2.imshow('img', img_with_keypoints)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [76]:
#Feature Matching

flann = cv2.FlannBasedMatcher(dict(algorithm=1, trees=5),dict(checks=50))

bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)


In [77]:
def match_fruit(descriptors):
    best_match = 0
    best_match_data = None

    for folder_path in os.listdir('Train/'):
        for img_path in os.listdir('Train/'+folder_path):
            img = cv2.imread('Train/'+folder_path+'/'+img_path)

            if img is not None:
                final_img, img_hist = preprocess_img(img)
                # if folder_path=='freshbanana':
                #     cv2.imshow('img', final_img)
                #     cv2.waitKey(0)
                #     cv2.destroyAllWindows()
                kp_sift, des_sift= extract_features(final_img)
                des_sift = np.float32(des_sift)
                # if folder_path == 'freshbanana':
                #     img_with_keypoints = cv2.drawKeypoints(final_img, kp_sift, None, color=(0,255,0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

                #     cv2.imshow('img', img_with_keypoints)
                #     cv2.waitKey(0)
                #     cv2.destroyAllWindows()

                matches = flann.knnMatch(descriptors,des_sift,k=2)

                good_matches = []

                curr_match = 0
                for i, (fn,sm) in enumerate(matches):
                    if fn.distance < 0.75 * sm.distance:
                        good_matches.append(fn)

                hist_score = cv2.compareHist(target_hist, img_hist, cv2.HISTCMP_CORREL)
                
                if (len(good_matches)>4):
                    src = np.float32([target_kp_sift[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                    dst = np.float32([kp_sift[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                    
                    _, mask = cv2.findHomography(src, dst, cv2.RANSAC, 5.0)

                    matches_mask = mask.ravel().tolist()
                    curr_match = sum(matches_mask)

                    final_match = (hist_score * 0.5) + (curr_match * 0.5)
                    if best_match < final_match:
                        best_match = final_match
                        best_match_data={
                            'img': img,
                            'kp': kp_sift,
                            'desc': des_sift,
                            'matches': good_matches,
                            'matches_mask':matches_mask
                        }
    
    result = cv2.drawMatches(
        cv2.resize(target_img,(256,256)),
        target_kp_sift,
        cv2.resize(best_match_data['img'],(256,256)),
        best_match_data['kp'],
        best_match_data['matches'],
        None,
        matchesMask=best_match_data['matches_mask'],
    )

    cv2.imshow('result',result)
    cv2.waitKey(0)

match_fruit(target_desc)

In [78]:
# def making_model():
#     desc = {}
#     for fruit in os.listdir('Training/'):
#         folder_path = os.path.join('Training', fruit)
        
#         if os.path.isdir(folder_path):
#             class_desc = []
            
#             for img_path in os.listdir(folder_path):
#                 img = cv2.imread(os.path.join(folder_path, img_path))
                
#                 if img is not None:
#                     final_img = preprocess_img(img)
#                     kp_sift, des_sift, kp_orb, des_orb = extract_features(final_img)
                    
#                     if des_sift is not None:
#                         class_desc.append(np.float32(des_sift))
            
#             if class_desc:
#                 desc[fruit] = np.vstack(class_desc)
    
#     with open('fruit_detection_model.pkl', 'wb') as file:
#         pickle.dump(desc, file)

# # making_model()

In [79]:
# descriptor_path = 'fruit_detection_model.pkl'

# with open(descriptor_path, 'rb') as file:
#     fruit_descriptors = pickle.load(file)

In [80]:
# def match_fruit_model(descriptors):
#     best_match = 0
#     best_fruit = "Unknown"
    
#     for fruit_class, stored_descriptors in fruit_descriptors.items():
#         if descriptors is not None:
#             matches = flann.knnMatch(descriptors, stored_descriptors, k=2)
            
#             good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]
            
#             if len(good_matches) > best_match:
#                 best_match = len(good_matches)
#                 best_fruit = fruit_class


In [81]:
# cap = cv2.VideoCapture(0)
# def img_detection_live():
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break

#         processed_frame = preprocess_img(frame)

#         kp_sift, des_sift, kp_orb, des_orb = extract_features(processed_frame)

#         frame_kp = cv2.drawKeypoints(frame, kp_sift, None, color=(0,255,0),
#                                                 flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

#         fruit_result = match_fruit_model(des_sift)

#         cv2.putText(frame_kp, fruit_result, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
#         cv2.imshow("Fruit Detection", frame_kp)

#         # Kalau mw keluar pencet q
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()

# # img_detection_live()